In [2]:
import pandas as pd
import numpy as np
import json
import pickle
import joblib

In [3]:
print(pd.__version__ )

2.1.2


In [4]:
!pipenv --version

pipenv, version 2023.10.3


In [5]:
!pipenv --venv

/home/majeedk/.local/share/virtualenvs/midterm_project-mvTjoeP0


In [58]:
# Add required libs into pipenv
!pipenv install scikit-learn==1.3.1 Flask gunicorn

Installing scikit-learn==1.3.1...
Resolving scikit-learn==1.3.1...
✔ Installation Succeeded
⠋ Installing scikit-learn...
Installing Flask...
Resolving Flask...
Added flask to Pipfile's [packages] ...
✔ Installation Succeeded..
⠋ Installing Flask...
Installing gunicorn...
Resolving gunicorn...
Added gunicorn to Pipfile's [packages] ...
✔ Installation Succeededrn...
⠋ Installing gunicorn...
Pipfile.lock (98d9d9) out of date, updating to (7bfb14)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠹ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (79d3f9418ecf7f0fc3fa89321a7bd9f80c9a7c0232e3081615808215b97bfb14)!
Installing dependencies from Pipfile.lock (7bfb14)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [40]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Read dictionary vectorizer
with open("models/dv_pkl.bin", "rb") as infile:
    dv = pickle.load(infile)

# Read the pre trained model    
with open("models/rf_modelv1_pkl.bin", "rb") as infile:
    model = pickle.load(infile)

### Perform prediction on a test person

In [27]:
# Test the model works 
person = {'HighBP': 'no',
  'HighChol': 'no',
  'CholCheck': 'yes',
  'BMI': 24,
  'Smoker': 'no',
  'Stroke': 'no',
  'HeartDiseaseorAttack': 'no',
  'PhysActivity': 'no',
  'Fruits': 'no',
  'Veggies': 'yes',
  'HvyAlcoholConsump': 'no',
  'AnyHealthcare': 'yes',
  'NoDocbcCost': 'no',
  'GenHlth': 'very_good',
  'MentHlth': 0,
  'PhysHlth': 0,
  'DiffWalk': 'no',
  'Sex': 'male',
  'Age': '55_59',
  'Education': 'high_school_graduate',
  'Income': '15k_20k'
  }
#'Diabetes_binary': 0

In [42]:
#df = pd.DataFrame(person,index=[0])
#df.columns = ["provide", "the", "list", "of", "feature names", "in", "order"]
#prediction = model.predict(df)
#person

In [37]:
person_data = dv.transform(person)

In [46]:
df = pd.DataFrame(person_data, columns=dv.feature_names_)
df

,Age_18_24,Age_25_29,Age_30_34,Age_35_39,Age_40_44,Age_45_49,Age_50_54,Age_55_59,Age_60_64,Age_65_69,...,PhysActivity_yes,PhysHlth,Sex_female,Sex_male,Smoker_no,Smoker_yes,Stroke_no,Stroke_yes,Veggies_no,Veggies_yes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0


In [48]:
print ('The probability that this client will get a credit is : {0}'.format(round( model.predict_proba(person_data)[:, 1][0],3)))

The probability that this client will get a credit is : 0.032


In [56]:
threshold = 0.14944631791961446

if model.predict_proba(person_data)[:, 1][0] >= threshold:
    print("This per likely has Diabetes")
else:
    print("No, this person doesn't have diabetes")

No, this person doesn't have diabetes


In [64]:
flask_code = """
import numpy as np
import joblib

from flask import Flask, request, jsonify

def predict_single(customer, dv, model):
    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred[0]

model = joblib.load('models/rf_modelv1.joblib')
dv = joblib.load('models/dv.joblib')

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():

    threshold = 0.14944631791961446
    patient = request.get_json()

    prediction = predict_single(patient, dv, model)
    has_diabetes = prediction >= threshold
    
    result = {
        'diabetes_probability': float(prediction),
        'has_diabetes': bool(has_diabetes),
    }

    return jsonify(result)


if __name__ == '__main__':
    app.run(debug=True)
    #, host='0.0.0.0', port=9696
"""

In [65]:
with open("app.py", "w") as outfile:
    outfile.write(flask_code)

In [66]:
! pipenv run gunicorn -b 0.0.0.0:9696 app:app

[2023-11-05 10:17:32 +0000] [5658] [INFO] Starting gunicorn 21.2.0
[2023-11-05 10:17:32 +0000] [5658] [INFO] Listening at: http://0.0.0.0:9696 (5658)
[2023-11-05 10:17:32 +0000] [5658] [INFO] Using worker: sync
[2023-11-05 10:17:32 +0000] [5660] [INFO] Booting worker with pid: 5660
/home/majeedk/.local/share/virtualenvs/midterm_project-mvTjoeP0/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/majeedk/.local/share/virtualenvs/midterm_project-mvTjoeP0/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.0 when using version 1.3.1. This might lead to break

In [67]:
# Added as I was getting a error - guess numpy isn't added to defauly pip env
!pipenv install numpy

Installing numpy...
Resolving numpy...
Added numpy to Pipfile's [packages] ...
✔ Installation Succeeded..
⠋ Installing numpy...
Pipfile.lock (7bfb14) out of date, updating to (59bce8)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠹ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (186853a9d21ffd21608c12dcb9838a7e4a7987e84ce6c5530ba255ea2359bce8)!
Installing dependencies from Pipfile.lock (59bce8)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
